# CSCI 5922 Exam 2
October 24, 2023

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Question 2:
```{math}
x =  [  [ 1   2   3] 
        [ 2   4   1]
        [ 4   3   2] ]

y = [0
    0
    1]

Assuming X is actually 3D

W1=[ [1, 1, 1]
    [1, 1, 1]
    [1, 1, 1] ]

W2 = [[1]
      [1]
      [1]]  

b = 0


Z1 = x*W1t + b
Z1 =  [  [ 1   2   3] 
        [ 2   4   1]
        [ 4   3   2] ]
*
    [[1, 1, 1]
    [1, 1, 1]
    [1, 1, 1] ]
+ 0

Z1=[[ 6 6 6]
    [ 7 7 7]
    [ 9 9 9]]

H1 = ReLU(Z1)
H1 = [[ 6 6 6]
    [ 7 7 7]
    [ 9 9 9]]

Z2 = H1*W2 + b
Z2 = [[ 6 6 6]
    [ 7 7 7]
    [ 9 9 9]]
    *
    [[1]
    [1]
    [1]]
    + 0
Z2 = [[18]
    [21]
    [27]]

Y^ = sigmoid(Z2)
Y^=[[0.9999999]
    [0.9999999]
    [0.9999999]]
Approximate 0.9999999 as 1
Y^=[[1]
    [1]
    [1]]

L = -y*log(y^) - (1-y)*log(1-y^)
L=[[18]
    [0]
    [0]]

dL/dW1 = dL/dy^ * dy^/dZ2 * dZ2/dH1 * dH1/dZ1 * dZ1/dW1

dL/dy^ = -y/y^ - (1-y)/(1-y^)
dy^/dZ2 = y^*(1-y^)
dZ2/dH1 = W2
dH1/dZ1 = 1 if Z1>0 else 0
dZ1/dW1 = x

dL/dy^ = [[-1]
        [0]
        [0]]

        *** COME BACK AND CALCULATE dL


In [3]:
#2a-c: Create and read in dataa
import numpy as np
import pandas as pd

np.set_printoptions(precision=6)

def get_xy(df):
   
    nrows = df.shape[0]
    ncols = df.shape[1]
   
    x = np.empty((nrows,ncols-1))
    y = np.empty((nrows,1))

    # separate labels
    if 'LABEL' in df.columns:
        y = np.array(df['LABEL'])
        x = np.array(df[df.columns.difference(['LABEL'])])
    elif 'label' in df.columns:
        y = np.array(df['label'])
        x = np.array(df[df.columns.difference(['label'])])
    else:
        raise Exception('There is no column with the title label in the data')
    return x, y

def binary_labels(y):
    y_values = set(y)
    y_value_0 = list(y_values)[0]
    y_value_1 = list(y_values)[1]
    y[y == y_value_0] = 0
    y[y == y_value_1] = 1
    y = y.reshape(-1, 1)
    return y.astype(int)

def normalize(x):
    xrows = x.shape[0]
    xcols = x.shape[1]
    xnorm = np.empty((xrows, xcols))
    for j, column in enumerate(x.T):
        max = column.max()
        min = column.min()
        for i, item in enumerate(column):
            xnorm[i,j] = (item - min)/(max - min)
    return xnorm.astype(float)


# create a function to one-hot-encode categorical data
def one_hot_encode(y):
    y_values = set(y)
    y_length = len(y_values)
    # convert values in y to integers
    for i, item in enumerate(y_values):
        y[y == item] = i

    # create new y array with one hot encoding
    y_new = np.zeros((len(y), y_length))
    for i, item in enumerate(y): 
        y_new[i, item] = 1    
    return y_new.astype(int)


In [4]:

df2 = pd.read_csv('A2_Exam_ILG.csv')
x2, y2 = get_xy(df2)
# x2 = normalize(x2)
y2 = one_hot_encode(y2)

print('The data is: \n', df2)
print('\nX is: \n', x2)
print('\nY one hot encoded is: \n', y2)


The data is: 
    experience  skill  athleticism  label
0           5      8            5      2
1           4      4            3      2
2           4      8            5      1
3           1      2            1      1
4           0      2            0      0
5           0      0            1      0

X is: 
 [[5 5 8]
 [3 4 4]
 [5 4 8]
 [1 1 2]
 [0 0 2]
 [1 0 0]]

Y one hot encoded is: 
 [[0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]]


The data is:   
    experience  skill  athleticism  label  
0           5      8            5      2  
1           4      4            3      2  
2           4      8            5      1  
3           1      2            1      1  
4           0      2            0      0  
5           0      0            1      0  
  
X is:   
 [[5 5 8]  
 [3 4 4]  
 [5 4 8]  
 [1 1 2]  
 [0 0 2]  
 [1 0 0]]  
  
Y one hot encoded is:   
 [[0 0 1]  
 [0 0 1]  
 [0 1 0]  
 [0 1 0]  
 [1 0 0]  
 [1 0 0]]  

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score

# activation function 1
def sigmoid(z):
    return 1/(1+np.exp(-z))

def sigmoid_derivative(z):
    return sigmoid(z) * (1-sigmoid(z))

# activation function 2
def relu(z):
    return np.maximum(0, z)
def relu_derivative(z):
    return np.where(z > 0, 1, 0)

# loss function
def cce(y, y_hat):
    return -np.sum(y*np.log(y_hat))

# output layer activation function
def softmax(z):
    return np.exp(z)/np.sum(np.exp(z), axis=1).reshape(-1, 1)


#2: Code the feedforward function
class NeuralNetworkCCE():
    # assumes one hidden layer
    # assumes one-hot-encoded y
    # assumes softmax activation on output layer
    # assumes cross-entropy loss

    def __init__(self, LR=0.1, epochs=10, activation='relu', input_layer_dim=3, hidden_layer_dim=2, output_layer_dim=4):
        self.LR = LR
        self.epochs = epochs
        # shape parameters
        self.hidden_layer_dim = hidden_layer_dim
        self.output_layer_dim = output_layer_dim
        self.input_layer_dim = input_layer_dim

        if activation == 'relu':
            self.activation = relu
            self.activation_derivative = relu_derivative
        elif activation == 'sigmoid': 
            self.activation = sigmoid
            self.activation_derivative = sigmoid_derivative
        
        if output_layer_dim == 1:
            self.output_activation = sigmoid
        else:
            self.output_activation = softmax

        # initialize with weights as ones and biases as zeros
        self.w1 = np.ones((self.hidden_layer_dim, self.input_layer_dim))
        self.b = np.zeros((1, self.hidden_layer_dim))
        self.w2 = np.ones((self.output_layer_dim, self.hidden_layer_dim))
        self.c = np.zeros((1, self.output_layer_dim))
    
    def randomize_weights(self):
        np.random.seed(44)
        self.w1 = np.random.rand(self.hidden_layer_dim, self.input_layer_dim)
        self.b = np.random.rand(1, self.hidden_layer_dim)
        self.w2 = np.random.rand(self.output_layer_dim, self.hidden_layer_dim)
        self.c = np.random.rand(1, self.output_layer_dim)

    def feed_forward(self, x):
        self.z1 = (x @ self.w1.T + self.b).astype(float)
        self.h = self.activation(self.z1).astype(float)
        self.z2 = (self.h @ self.w2.T + self.c).astype(float)
        self.y_hat = self.output_activation(self.z2).astype(float)
        self.output = np.argmax(self.y_hat, axis=1).reshape(-1, 1)
        return
   
    def calculate_loss(self, y):
        self.loss = cce(y, self.y_hat).astype(float)
        return
    
    def print_params(self, y):
        print('\nW1 is: \n', self.w1.T) # shape is (2x3)
        print('\nB is: \n', self.b) # shape is (1x2)
        print('\nZ1 is: \n', self.z1) # shape is (17x2)
        print('\nH is: \n', self.h) # shape is (17x2)
        print('\nW2 is: \n', self.w2.T) # shape is (4x2)
        print('\nZ2 is: \n', self.z2) # shape is (17x4)
        print('\nC is: \n', self.c) # shape is (1x4)
        print('\ny^ is: \n', self.y_hat) # shape is (17x4)
        print('\ny^-y is: \n', self.y_hat - y) # shape is (17x4)
        print('\noutput is: \n', self.output) # shape is (17x1)
        print('\nL is: \n', self.loss) # shape is (1x1)
        

    def backpropogate(self, x, y):
        self.dLdC = (self.y_hat - y).astype(float) # shape is (17x4) 
        self.dLdW2 = (self.dLdC.T @ self.h).astype(float) # (4x17)*(17x2) = (4x2)
        self.dLdB = (self.dLdC @ self.w2 * self.activation_derivative(self.z1)).astype(float) # (17x4) @ (4x2) * (17x2) = (17x2)
        self.dLdW1 = (self.dLdB.T @ x).astype(float) # (2x17)*(17x3) = (2x3)
    
    def print_derivatives(self):
        print('\ndLdC is: \n', self.dLdC) # shape is (17x4)
        print('\ndLdW2 is: \n', self.dLdW2) # shape is (4x2)
        print('\ndLdB is: \n', self.dLdB) # shape is (17x2)
        print('\ndLdW1 is \n', self.dLdW1) # shape is (2x3)

    def print_new_params(self):
        print('\nNew W1 is: \n', self.w1.T)
        print('\nNew B is: \n', self.b)
        print('\nNew W2 is: \n', self.w2.T)
        print('\nNew C is: \n', self.c)

    def update_params(self):
        self.c = (self.c - self.LR * self.dLdC.mean(axis=0)).astype(float) # (1x4)-LR*(17x4).mean = (1x4)
        self.w2 = (self.w2 - self.LR * self.dLdW2).astype(float) # (4x2)-LR*(4x2).mean = (4x2) 
        self.b = (self.b - self.LR * self.dLdB.mean(axis=0)).astype(float) # (1x2)-LR*(17x2).mean = (1x2)
        self.w1 = (self.w1 - self.LR * self.dLdW1).astype(float) # (2x3)-LR*(2x3) = (2x3)
        return

    def train(self, x, y, verbose=False, plot=True):
        if verbose:
            print('Preparing to train Neural Network...')
            print('X is: \n', x)
            print('\ny is: \n', y)
        self.randomize_weights()
        self.loss_by_epoch = []
        for e in range(self.epochs):
            if verbose:
                print('###############################################################\nNow running epoch', e+1, 'of', self.epochs, 'total epochs.')
                print('Feeding forward...')
            self.feed_forward(x)
            if verbose:
                print('Calculating loss...')
            self.calculate_loss(y)
            if verbose:
                print('The parameters are: ')
                self.print_params(y)
                print('\nPerforming backpropogation...')
            self.backpropogate(x, y)
            self.update_params()
            if verbose:
                print('The derivatives are: ')
                self.print_derivatives()
                self.print_new_params()
                print('\nFinished epoch', e+1, 'with loss', self.loss)
            self.loss_by_epoch.append(self.loss)
        if verbose:
            print('Training complete! Loss by epoch is: \n', self.loss_by_epoch)
        if plot:
            y = self.loss_by_epoch
            x = range(self.epochs)
            plt.plot(x, y)
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Loss by Epoch')
            plt.show()

    # multilabel confusion matrix
    def plot_confusion_matrix(self, y):
        y_pred = self.output
        y = np.argmax(self.y_hat, axis=1).reshape(-1, 1)
        self.cm = confusion_matrix(y, y_pred)
        classes = ['0: Bad', '1: Okay', '2: Good', '3: Great']
        display = ConfusionMatrixDisplay(self.cm, display_labels=classes)
        display.ax_.set_title('Confusion Matrix for Player Ability Prediction')
        display.plot()
        # calculate accuracy
        self.accuracy = accuracy_score(y, y_pred)
        print('Accuracy is: ', self.accuracy)
    
    def predict(self, x, y):
        self.feed_forward(x)
        self.plot_confusion_matrix(y)

In [14]:
# test code with verbose mode
nn1 = NeuralNetworkCCE(
    LR=1,
    epochs=1, 
    activation='sigmoid', 
    input_layer_dim=3, 
    hidden_layer_dim=2, 
    output_layer_dim=3
)
nn1.train(x2, y2, verbose=True, plot=False)

Preparing to train Neural Network...
X is: 
 [[5 5 8]
 [3 4 4]
 [5 4 8]
 [1 1 2]
 [0 0 2]
 [1 0 0]]

y is: 
 [[0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]]
###############################################################
Now running epoch 1 of 1 total epochs.
Feeding forward...
Calculating loss...
The parameters are: 

W1 is: 
 [[0.834842 0.360501]
 [0.104796 0.359311]
 [0.74464  0.609238]]

B is: 
 [[0.39378  0.409073]]

Z1 is: 
 [[11.049095  8.882038]
 [ 6.296052  5.364772]
 [10.944299  8.522727]
 [ 2.822699  2.347361]
 [ 1.883061  1.627549]
 [ 1.228622  0.769573]]

H is: 
 [[0.999984 0.999861]
 [0.99816  0.995343]
 [0.999982 0.999801]
 [0.94389  0.912724]
 [0.867962 0.835834]
 [0.773577 0.683429]]

W2 is: 
 [[0.509902 0.960526 0.427652]
 [0.710148 0.456621 0.113464]]

Z2 is: 
 [[1.437843 2.374541 1.484443]
 [1.433704 2.370726 1.483151]
 [1.437799 2.374512 1.484436]
 [1.34736  2.280872 1.450568]
 [1.254041 2.172832 1.409373]
 [1.097683 2.012581 1.351717]]

C is: 
 [[0.217899 0

W1 is:   
 [[0.834842 0.360501]  
 [0.104796 0.359311]  
 [0.74464  0.609238]]  
  
W2 is:   
 [[0.509902 0.960526 0.427652]  
 [0.710148 0.456621 0.113464]]  
  
B is:   
 [[0.39378  0.409073]]  
  
   
C is:   
 [[0.217899 0.957472 0.943351]]  
  
Z1 is:   
 [[11.049095  8.882038]  
 [ 6.296052  5.364772]  
 [10.944299  8.522727]  
 [ 2.822699  2.347361]  
 [ 1.883061  1.627549]  
 [ 1.228622  0.769573]]  
  
H is:   
 [[0.999984 0.999861]  
 [0.99816  0.995343]  
 [0.999982 0.999801]  
 [0.94389  0.912724]  
 [0.867962 0.835834]  
 [0.773577 0.683429]]  
  
Z2 is:   
 [[1.437843 2.374541 1.484443]  
 [1.433704 2.370726 1.483151]  
 [1.437799 2.374512 1.484436]  
 [1.34736  2.280872 1.450568]  
 [1.254041 2.172832 1.409373]  
 [1.097683 2.012581 1.351717]]  
  
  
y^ is:   
 [[0.217427 0.554774 0.227799]  
 [0.217247 0.554494 0.228259]  
 [0.217423 0.554773 0.227803]  
 [0.214954 0.546721 0.238325]  
 [0.213936 0.536178 0.249887]  
 [0.208954 0.521659 0.269387]]  
  
y^-y is:   
 [[ 0.217427  0.554774 -0.772201]  
 [ 0.217247  0.554494 -0.771741]  
 [ 0.217423 -0.445227  0.227803]  
 [ 0.214954 -0.453279  0.238325]  
 [-0.786064  0.536178  0.249887]  
 [-0.791046  0.521659  0.269387]]  
  
dLdC is:   
 [[ 0.217427  0.554774 -0.772201]  
 [ 0.217247  0.554494 -0.771741]  
 [ 0.217423 -0.445227  0.227803]  
 [ 0.214954 -0.453279  0.238325]  
 [-0.786064  0.536178  0.249887]  
 [-0.791046  0.521659  0.269387]]  
  
dLdW2 is:   
 [[-0.439626 -0.350436]  
 [ 1.1041    1.052424]  
 [-0.664474 -0.701988]]  
  
dLdB is:   
 [[ 4.985111e-06  4.443842e-05]  
 [ 5.755481e-04  1.482784e-03]  
 [-3.873533e-06 -4.582692e-06]  
 [-1.185601e-02 -2.173579e-03]  
 [ 2.533444e-02 -3.911183e-02]  
 [ 3.729315e-02 -6.339030e-02]]  
  
dLdW1 is   
 [[ 0.027169 -0.009544  0.029268]  
 [-0.060916  0.003961 -0.076321]]  
  
Average Loss is:   
 7.2572989286716245  
  
New W1 is:   
 [[0.807673 0.421417]  
 [0.11434  0.355349]  
 [0.715373 0.685559]]  

In [15]:
df3 = pd.read_csv('A2_Exam2_ILG.csv')
x3, y3 = get_xy(df3)
# x3 = normalize(x3)
# y3 = one_hot_encode(y3)

print('The data is: \n', df3)
print('\nX is: \n', x3)
print('\nY is: \n', y3)


The data is: 
    x1  x2  x3  label
0   1   2   3      0
1   2   4   1      1
2   4   3   2      1

X is: 
 [[1 2 3]
 [2 4 1]
 [4 3 2]]

Y is: 
 [0 1 1]


In [16]:
# test code with verbose mode
nn2 = NeuralNetworkCCE(
    LR=1,
    epochs=1, 
    activation='sigmoid', 
    input_layer_dim=3, 
    hidden_layer_dim=3, 
    output_layer_dim=1
)
nn2.train(x3, y3, verbose=True, plot=False)

Preparing to train Neural Network...
X is: 
 [[1 2 3]
 [2 4 1]
 [4 3 2]]

y is: 
 [0 1 1]
###############################################################
Now running epoch 1 of 1 total epochs.
Feeding forward...
Calculating loss...
The parameters are: 

W1 is: 
 [[0.834842 0.360501 0.39378 ]
 [0.104796 0.359311 0.409073]
 [0.74464  0.609238 0.509902]]

B is: 
 [[0.710148 0.960526 0.456621]]

Z1 is: 
 [[3.988504 3.867364 3.198253]
 [3.543657 3.72801  3.390373]
 [5.853186 4.698939 4.278762]]

H is: 
 [[0.98181  0.979515 0.960768]
 [0.971905 0.976524 0.967402]
 [0.997137 0.990977 0.98633 ]]

W2 is: 
 [[0.427652]
 [0.113464]
 [0.217899]]

Z2 is: 
 [[1.697834]
 [1.694704]
 [1.711259]]

C is: 
 [[0.957472]]

y^ is: 
 [[1.]
 [1.]
 [1.]]

y^-y is: 
 [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]

output is: 
 [[0]
 [0]
 [0]]

L is: 
 -0.0

Performing backpropogation...


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1 is different from 3)